In [1]:
import numpy as np
import pandas as pd

In [2]:
def load_all_ancients(replace_nan=False, normalize=False):
  x = np.load('data/genotype_ancient_refs.npy')
  
  #x_s = select_snps(x)

  # normalization
  genomean_path = 'data/genomean.csv'
  genomean = pd.read_csv(genomean_path, header=0)
  genomean = genomean['x'].values
  snp_drift = np.sqrt((genomean / 2) * (1 - genomean/2)) 

  # replace nans by genomean
  if replace_nan:
    x_s_i = np.nan_to_num(x, nan=genomean)
  else:
    x_s_i = x

  # normalization
  if normalize:
    x_s_i_c = (x_s_i - genomean) / snp_drift
  else:
    x_s_i_c = x_s_i

  return x_s_i_c

In [3]:
def select_snps(x):
  present_modern_snps_path = 'data/SNPs_mwe.csv'
  present_modern_snps = pd.read_csv(present_modern_snps_path, header=0)
  modern_snps_indices = present_modern_snps['x'].values
  return np.array([i[modern_snps_indices-1] for i in x])

In [4]:
def load_and_preprocess_data(replace_nan=True):
  x = parse_geno_file_path('data/modern_genotypes.geno') 
  x_s = select_snps(x)
  return x_s

In [5]:
def get_genomean():
  genomean_path = 'data/genomean.csv'
  genomean = pd.read_csv(genomean_path, header=0)
  genomean = genomean['x'].values
  return genomean

In [6]:
def get_snp_drift():
  genomean_path = 'data/genomean.csv'
  genomean = pd.read_csv(genomean_path, header=0)
  genomean = genomean['x'].values
  snp_drift = np.sqrt((genomean / 2) * (1 - genomean/2))
  return snp_drift

In [ ]:
def parse_geno_file_path(file_path):
    with open(file_path, 'r') as file:
        geno_lines = file.readlines()
    
    geno_data = []
    for idx, line in enumerate(geno_lines):
        #if idx % 100000 == 0 and idx > 0:
        #    print(f"Parsed {idx} lines of 540248")
        geno_data.append(list(map(int, line.strip())))
    
    #print("transposing")
    d = np.array(geno_data, dtype=np.uint8).T
    return(np.where(d==9., np.nan, d))

# Variance raw data

In [7]:
a_nn = load_all_ancients(replace_nan=False, normalize=False)

In [9]:
var_a_nn = np.nanvar(a_nn, axis=0)

In [15]:
m_nn = load_and_preprocess_data(replace_nan=False)

Parsed 100000 lines of 540248
Parsed 200000 lines of 540248
Parsed 300000 lines of 540248
Parsed 400000 lines of 540248
Parsed 500000 lines of 540248
transposing


In [18]:
var_m_nn = np.nanvar(m_nn, axis=0)

# Variance normalized data

In [20]:
m_norm = (m_nn - get_genomean()) /get_snp_drift()
a_norm = (a_nn - get_genomean()) /get_snp_drift()

In [21]:
var_a_norm = np.nanvar(a_norm, axis=0)
var_m_norm = np.nanvar(m_norm, axis=0)

Problem: some of the features in the normalized ancients have an exreme high-variance. They are considered as outliers and replaced by the mean variance of the other features. Otherwise, they "destroy" the factor calculation...

In [34]:
var_smaller_10 = np.where(var_a_norm<10)
var_mean = np.mean(var_a_norm[var_smaller_10])
var_a_norm[var_a_norm > 10] = var_mean

In [36]:
V = np.load('data/eigenvectors.npy')
Lambda = np.load('data/eigenvalues.npy')
f_in = var_a_norm/var_m_norm
f_out = ((V.T * f_in) @ V)

In [60]:
np.save('paper_figures/variance_estimation/var_a_nn.npy', var_a_norm)
np.save('paper_figures/variance_estimation/var_m_nn.npy', var_m_norm)
np.save('paper_figures/variance_estimation/factors.npy', np.diag(f_out))